## CLUSTERING AS A PRE-PROCESSING STEP (k=10)-

#### Since the models directly done on the Kaggle dataset didn't perform well, we will do another clustering for the hotels and check the models on the newly clustered data.
***
### 1. Load the cleaned original train dataset-

In [18]:
import pandas as pd 
import numpy as np 

path_train_final ='F:/AML/Project/clean dataset/FinalExpedia_set_cleaned/Train_set_cleaned.csv'

train_set=pd.read_csv(path_train_final)

In [19]:
train_set.shape

(2512931, 39)

### 2. Clustering-
#### Elbow method which we saw earlier gave optimum k=20.
***

##### KMeans algorithm is used for creating the optimum no.of clusters (k=20) and we have seen this performance already. Now, let us check with k=10 to understand how accuracy is with diff. cluster sizes.

In [20]:
from sklearn.cluster import KMeans

Kmean = KMeans(n_clusters=10).fit(train_set[['srch_destination_id', 'srch_destination_type_id', 'cnt',
       'hotel_continent', 'hotel_country', 'hotel_market']])

new_clusters=Kmean.labels_
train_set["new_hotel_cluster"]=new_clusters #update train_set with new hotel clusters
train_set["new_hotel_cluster"].value_counts() #new cluster distribution

C:\Users\User-\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0    954068
4    542922
1    250698
3    203284
9    200150
5    149606
2     97591
8     46057
7     46001
6     22554
Name: new_hotel_cluster, dtype: int64

#### Split this train set to train and test datasets in 80:20 ratio.

In [21]:
predictors = [c for c in train_set.columns if c not in ["new_hotel_cluster"]]

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_set[predictors], train_set["new_hotel_cluster"], test_size=0.2)

### 3. Using models-
#### Apply the classification models on this clustered data.

#### RANDOM FOREST-
***
##### a. with 50 binary trees-

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

clf = RandomForestClassifier(n_estimators=50, min_weight_fraction_leaf=0.1)
clf.fit(X_train,y_train)
preds = clf.predict(X_test)

In [24]:
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:",precision_score(y_test, preds, average="micro"))
print("Recall:",recall_score(y_test, preds,  average="micro"))
print("F1 Score:",f1_score(y_test, preds,  average="micro"))

Accuracy: 0.5999617976589128
Precision: 0.5999617976589128
Recall: 0.5999617976589128
F1 Score: 0.5999617976589128


##### b. with 100 binary trees-

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

clf = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf=0.1)
clf.fit(X_train,y_train)
preds = clf.predict(X_test)

In [26]:
from sklearn.metrics import f1_score,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy:", accuracy_score(y_test, preds))
print("Precision:",precision_score(y_test, preds, average="micro"))
print("Recall:",recall_score(y_test, preds,  average="micro"))
print("F1 Score:",f1_score(y_test, preds,  average="micro"))

Accuracy: 0.6102246974155718
Precision: 0.6102246974155718
Recall: 0.6102246974155718
F1 Score: 0.6102246974155718


#### KERAS-
***
##### Load the required libraries-

In [27]:
!pip install tensorflow

In [28]:
!pip install scikeras
import tensorflow as tf
import keras
from scikeras.wrappers import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import KFold,train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

import os
import gc
import zipfile
import io

import pickle
import time
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

##### Consider the initial train_set-

In [36]:
train_set['new_hotel_cluster'].replace([0,1,2,3,4,5,6,7,8,9],  
                               ['zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine'], 
                               inplace=True)
train_set['new_hotel_cluster'].value_counts()

zero     954068
Four     542922
One      250698
Three    203284
Nine     200150
Five     149606
Two       97591
Eight     46057
Seven     46001
Six       22554
Name: new_hotel_cluster, dtype: int64

In [37]:
X_train_k = train_set.drop(['new_hotel_cluster'], axis=1)
y_train_k = train_set['new_hotel_cluster']

In [38]:
from sklearn.preprocessing import LabelEncoder

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train_k)
encoded_y = encoder.transform(y_train_k)
# convert integers to dummy variables (i.e. one hot encoded)
y = keras.utils.to_categorical(encoded_y)

In [39]:
X_traink, X_testk, y_traink, y_testk = train_test_split(X_train_k, y, test_size=0.2, random_state=1)

In [40]:
# scales data between 0 and 1
X_traink = keras.utils.normalize(X_traink, axis=1)  
X_testk = keras.utils.normalize(X_testk, axis=1)

##### The Keras model-

In [41]:
def c_model():

  model = Sequential()
  model.add(Dense(2048, activation='relu', input_dim=39))
  # model.add(Dense(1024, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(512, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(256, activation='relu'))
  # model.add(Dropout(0.25))
  # model.add(Dense(128, activation='relu'))
  # model.add(Dropout(0.25))
  model.add(Dense(64, activation='relu'))
  # model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

##### a. For epochs=15, batch_size=20000

In [42]:
model = KerasClassifier(build_fn=c_model, epochs=15, batch_size=20000)
model.fit(X_traink, y_traink)

C:\Users\User-\anaconda3\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/15


101/101 [==============================] - 173s 1s/step - loss: 1.5032 - accuracy: 0.4463
Epoch 2/15
101/101 [==============================] - 122s 1s/step - loss: 0.6865 - accuracy: 0.7402
Epoch 3/15
101/101 [==============================] - 128s 1s/step - loss: 0.4030 - accuracy: 0.8627
Epoch 4/15
101/101 [==============================] - 136s 1s/step - loss: 0.4424 - accuracy: 0.8460
Epoch 5/15
101/101 [==============================] - 132s 1s/step - loss: 0.1952 - accuracy: 0.9464
Epoch 6/15
101/101 [==============================] - 131s 1s/step - loss: 0.4803 - accuracy: 0.8520
Epoch 7/15
101/101 [==============================] - 131s 1s/step - loss: 0.2096 - accuracy: 0.9429
Epoch 8/15
101/101 [==============================] - 131s 1s/step - loss: 0.1305 - accuracy: 0.9695
Epoch 9/15
101/101 [==============================] - 132s 1s/step - loss: 0.1252 - accuracy: 0.9605
Epoch 10/15
101/101 [==============================] - 128s 1s/step - loss: 0.1272 - accu

KerasClassifier(
	model=None
	build_fn=<function c_model at 0x0000025A36DB9580>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=20000
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=15
	class_weight=None
)

In [43]:
pred=model.predict(X_testk)

26/26 [==============================] - 13s 386ms/step


In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import f1_score


print("F1 Score:", f1_score(pred,y_testk, average="micro"))
print("Accuracy:", accuracy_score(pred,y_testk))
print("Precision:",precision_score(pred,y_testk, average="micro"))
print("Recall:",recall_score(pred,y_testk,  average="micro"))


F1 Score: 0.9826955333106507
Accuracy: 0.9826955333106507
Precision: 0.9826955333106507
Recall: 0.9826955333106507


##### We have a very high accuracy here. This might be because the clusters aren't able to address the complexity in the data and are oversimplifying the dataset. Hence, k=10 is not a good approach.